In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk import word_tokenize
nltk.download('punkt')
import gensim
from gensim.models import Word2Vec
import multiprocessing

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [38]:
data = open('/content/gdrive/My Drive/Colab/shak/alllines.txt').read()

In [39]:
tokens = word_tokenize(data)

In [40]:
tokens = [w.lower() for w in tokens if w.isalpha()]

In [41]:
in_seq = 101
lines = []

for i in range(in_seq,len(tokens)):
  seq = tokens[i-in_seq:i]
  line = ' '.join(seq)
  lines.append(line)
  if i>200000:
    break

In [42]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [43]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [44]:
sequences = np.array(sequences)

In [45]:
x,y = sequences[:,:-1],sequences[:,-1]

In [46]:
vocab_size = len(tokenizer.word_index) + 1

In [47]:
y = to_categorical(y,num_classes=vocab_size)

In [48]:
seq_len = x.shape[1]

In [49]:
model = Sequential()
model.add(Embedding(vocab_size,100,input_length = seq_len))
model.add(LSTM(100,return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100,activation = 'relu'))
model.add(Dense(vocab_size,activation='softmax'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 100)          1169800   
_________________________________________________________________
lstm_4 (LSTM)                (None, 100, 100)          80400     
_________________________________________________________________
lstm_5 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_4 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_5 (Dense)              (None, 11698)             1181498   
Total params: 2,522,198
Trainable params: 2,522,198
Non-trainable params: 0
_________________________________________________________________


In [50]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [51]:
model.fit(x,y,batch_size=256,epochs=1)

671/781 [========================>.....] - ETA: 10s - loss: 6.7475 - accuracy: 0.0372

KeyboardInterrupt: ignored

In [ ]:
model.save('/content/gdrive/My Drive/Colab/shak/shak63.h5')

In [ ]:
def generate(model,tokenizer,text_seq_length,seed_text,n_words):
  text = []

  for _ in range(n_words):
    encoded = tokenizer.texts_to_sequences([seed_text])[0]
    encoded = pad_sequences([encoded],maxlen = text_seq_length,truncating='pre')

    y_predict = model.predict_classes(encoded)

    predicted_word = ''
    for word,index in tokenizer.word_index.items():
      if index == y_predict:
        predicted_word = word
        break

    seed_text = seed_text + ' ' + predicted_word
    text.append(predicted_word)
  return ' '.join(text)

In [ ]:
lines[1234]

'i to do with a buff jerkin why what a pox have i to do with my hostess of the tavern well thou hast called her to a reckoning many a time and oft did i ever call for thee to pay thy part no i give thee thy due thou hast paid all there yea and elsewhere so far as my coin would stretch and where it would not i have used my credit yea and so used it that were it not here apparent that thou art heir apparent but i prithee sweet wag shall there be gallows standing'

In [ ]:
generate(model,tokenizer,seq_len,lines[1234],50)

'in base ways there does the city are not the giddy multitude leaving their furred thoughts villain would begin our fill against our nation and with the pride of our that her arms hath a book to this rome slain no antony and yet must wit awhile and yet are'

In [ ]:
import tensorflow as tf 
model = tf.keras.models.load_model('/content/gdrive/My Drive/Colab/shak/shak.h5')

In [ ]:
model.fit(x,y,batch_size=256,epochs=50)

Epoch 1/50
781/781 [==============================] - 35s 44ms/step - loss: 2.0472 - accuracy: 0.5285
Epoch 2/50
781/781 [==============================] - 35s 45ms/step - loss: 2.0328 - accuracy: 0.5312
Epoch 3/50
781/781 [==============================] - 35s 45ms/step - loss: 2.0158 - accuracy: 0.5351
Epoch 4/50
781/781 [==============================] - 34s 44ms/step - loss: 2.0060 - accuracy: 0.5356
Epoch 5/50
781/781 [==============================] - 34s 44ms/step - loss: 1.9948 - accuracy: 0.5392
Epoch 6/50
781/781 [==============================] - 34s 44ms/step - loss: 1.9778 - accuracy: 0.5424
Epoch 7/50
781/781 [==============================] - 34s 44ms/step - loss: 1.9661 - accuracy: 0.5435
Epoch 8/50
781/781 [==============================] - 35s 44ms/step - loss: 1.9516 - accuracy: 0.5471
Epoch 9/50
781/781 [==============================] - 35s 44ms/step - loss: 1.9426 - accuracy: 0.5487
Epoch 10/50
781/781 [==============================] - 35s 44ms/step - loss: 1.918